In [55]:
import json
import os
import requests

def read_chunks(chunk_dir):
    chunks = []
    for filename in os.listdir(chunk_dir):
        if filename.endswith(".json"):
            with open(os.path.join(chunk_dir, filename), 'r', encoding='utf-8') as f:
                chunk_data = json.load(f)
                chunks.append(chunk_data)
    return chunks

In [54]:
count_chunks = len(read_chunks('./chunks'))

In [74]:
def llm_request(prompt: str, api_key: str) -> str:
  try:
    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        },
        data=json.dumps({
            "model": "deepseek/deepseek-r1-0528:free",
            "messages": [{"role": "user", "content": prompt}],
        })
    )

    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"Ошибка {response.status_code}: {response.text}"

  except Exception as e:
    return f"Ошибка при запросе: {str(e)}"

In [ ]:
def process_text_method_1(chunk_content, api_key):
  ontology = '''
@prefix comcore:  .
@prefix dc:  .
@prefix dcmtype:  .
@prefix dcterms:  .
@prefix owl:  .
@prefix rdf:  .
@prefix rdfs:  .
@prefix xsd:  .

 a owl:Ontology;
     dc:title "Самая общая онтология суперклассов для описания бизнес-процессов сервисов компании" ;
     rdfs:comment """
 Данная онтология описывает суперклассы и их отношения, которые используются при анализе текстовых документов/чанков. Содержит описание сервисов и их вариантов использования - основных классов и составляющих бизнес-процессов: Процессов, Акторов, входных Ресурсов и результирующих продуктов (артефактов), Событий, определяющих цепочку простых Процессов (Актов). """ ;
     rdfs:isDefinedBy ;
     rdfs:seeAlso , ;
     owl:imports , , , ;
     owl:versionIRI ;
     owl:versionInfo "
".

comcore:Process a rdfs:Class;
  rdfs:label "Процесс"@ru;
  rdfs:label "Process"@en;
  rdfs:comment "Процесс, который является подпроцессом Бизнес-процессов";
  dc:description """
Процесс-это продолжительное взаимодействие или работа, выполняемая или инициируемая Акторами (Агентами в роли Акторов) в зависимости от происходящих Событий (сигналов для Акторов, Процессов или других Событий) с использованием Ресурсов с целью создания других Ресурсов и формирования исходов, условий наступления Событий, которые могут стать сигналом для инициации очередного Процесса; """.

comcore:Resource rdfs:subClassOf rdfs:Resource;
  rdfs:label "Ресурс"@ru;
  rdfs:label "Resource"@en;
  rdfs:comment "Это всё, что используется Акторами для реализации Процесса";
  dc:description """
Ресурс - это то, что используется/тратится при реализации процесса или создается в результате процесса. Как правило, за обеспечение Процесса данным Ресурсом отвечает один или несколько Акторов.
Пример: В Процессе подключения Клиента к платежной системе обрабатываются Заявка Клиента на подключение в которой должен обязательно быть его ИНН.
Ресурсы: Заявка Клиента на подключение, ИНН Клиента.""".

comcore:Agent rdfs:subClassOf dcmtype:Agent;
  rdfs:label "Агент"@ru;
  rdfs:label "Agent"@en;
  rdfs:comment "Это сущность, которая активно участвует в Процессе(Бизнес-процессе) и выполняет какую-то роль, реализует определенную функцию";
  dc:description """
Агент - это сущность, которая участвует в Процессе и выполняет какую-то роль, реализует функцию или отвечает за Событие. Агенты взаимодействуют в Процессе. Они инициализируют Подпроцессы, отдельные Акты, результаты которых используют другие Агенты.
Например, для реализации Процесса сопровождения в качестве Агента со стороны компании
может выступать Агент - Сотрудник службы сопровождения, а может выступать Агент - искусственный интеллект или просто какой-то микросервис.
Агент способен получать сообщения и выполнять на их основе какие-то действия, реализовать функцию.
Агент использует для реализации своей функции Ресурсы. Он может быть ответственным за обеспечение Процесса даннным Ресурсом.
Пример: При подключении Клиента к платежной системе задействуются Сотрудник службы безопасности компании, Сотрудник отдела по обслуживанию, сервис мониторинга заполненности ЛК.
Агенты: Сотрудник службы безопасности компании, Сотрудник отдела по обслуживанию, Сервис мониторинга заполненности ЛК.""".

comcore:Event rdfs:subClassOf dcmtype:Event;
  rdfs:label "Событие"@ru;
  rdfs:label "Event"@en;
  rdfs:comment "Событие - стартовое условие запуска следующего Процесса, может быть результатом предыдущего Процесса или инициироваться Агентом (Клиентом, Оператором, ПО)";
  dc:description """
Это описание какого-то исхода в результате реализации Процесса, или События, инициирующего Процесс.
Пример: Проверка MRM задачи может иметь три статуса (Согласовано, Отказано, Нет решения), при отсутствии решения отправляется запрос дополнительных документов.
События: Статус Согласовано, Статус Отказано, Статус Нет решения, Отправка запроса дополнительных документов.""".

comcore:isResultOf rdf:type owl:ObjectProperty;
  owl:inverseOf comcore:hasResult;
  rdf:type owl:IrreflexiveProperty;
  rdfs:label "результат"@ru;
  rdfs:label "result"@en;
  rdfs:domain comcore:Resource;
  rdfs:domain comcore:Event;
  rdfs:range comcore:Process;
  rdfs:comment """
Указывает что Событие или Ресурс (субъект) является результатом Процесса (объект).
Пример: Регистрация в форме может закончиться неуспешно если ИНН или Email уже зарегистрирован.
Субъекты: Успешная регистрация, ИНН уже зарегистрирован, Email уже зарегистрирован.""".

comcore:canBeResourceFor rdf:type owl:ObjectProperty;
  rdfs:label "может быть Ресурсом"@ru;
  rdfs:label "может использоваться как Ресурс в"@ru;
  rdfs:label "can be resource for"@en;
  rdfs:label "can be resource in"@en;
  rdfs:domain comcore:Resource;
  rdfs:range comcore:Process;
  rdfs:comment """Указывает что Ресурс (субъект) может использоваться в качестве входного Ресурса данного Процесса (объект). Например, ответ на секретный вопрос может использоваться при аутентификации""" .

comcore:isResourceOf rdf:type owl:ObjectProperty;
  owl:inverseOf comcore:hasResource;
  rdfs:label "является Ресурсом"@ru;
  rdfs:label "is Resource of"@en;
  rdfs:domain comcore:Resource;
  rdfs:range comcore:Process;
  rdfs:comment """Указывает что Ресурс (субъект) является необходимым входным Ресурсом данного Процесса (объект). Например, логин или пароль при аутентификации""" .

comcore:isActorOf rdf:type owl:ObjectProperty;
  owl:inverseOf comcore:hasActor;
  rdfs:label "является Актором"@ru;
  rdfs:label "is Actor of"@en;
  rdfs:comment "Указывает Агента (субъект), который в какой-то роли принимает участие в Процессе (объект)";
  rdfs:domain comcore:Agent;
  rdfs:range comcore:Process.

comcore:Initiates rdfs:subPropertyOf comcore:;
  rdfs:label "инициирует"@ru;
  rdfs:label "initiates"@en;
  rdfs:comment "Указывает что субъект инициирует данный Процесс.";
  rdfs:domain comcore:Event;
  rdfs:domain comcore:Agent;
  rdfs:range comcore:Process.

comcore:isPartOf rdfs:subPropertyOf dcterms:isPartOf;
  owl:inverseOf comcore:hasPart;
  rdf:type owl:IrreflexiveProperty;
  rdf:type owl:TransitiveProperty;
  rdfs:label "является частью"@ru;
  rdfs:label "is part of"@en;
  rdfs:domain comcore:Process;
  rdfs:domain comcore:Resource;
  rdfs:range comcore:Process;
  rdfs:range comcore:Resource;
  rdfs:comment "Указывает, что субъект является частью для данного объекта".

comcore:isResponsibleFor rdfs:subPropertyOf dcterms:contributor;
  rdfs:label "ответственный"@ru;
  rdfs:label "responsible"@en;
  rdfs:comment "Указывает, что Агент (субъект) отвечает за наличие/создание данного Ресурса (объект) или является ответственным за проведение Процесса";
  rdfs:domain comcore:Agent;
  rdfs:range comcore:Resource;
  rdfs:range comcore:Process.

comcore:relation rdfs:subPropertyOf dcterms:relation;
  rdf:type owl:IrreflexiveProperty;
  rdfs:label "связан с"@ru;
  rdfs:label "relation"@en;
  rdfs:comment "Указывает, что одна сущность (субъект) как-то связана с другой сущностью (объект). Отношение используется, если не подходят вышеперечисленные отношения".
'''
  prompt_E = f'''Извлеките из ТЕКСТА все сущности классов, описанных в ОНТОЛОГИИ и сформируй их описание, с использованием RDF-модели в формате Turtle.
ТРЕБОВАНИЯ:
- извлекаются ВСЕ сущности (субъекты и объекты), которые подходят под описание в онтологии;
при этом для каждой извлеченной сущности должны быть созданы минимум два триплета - триплет, который описывает тип сущности и триплет с именем сущности:
(идентификатор_сущности, rdf:type, ont_name:class_name) ;
(идентификатор_сущности, rdfs:label, "имя сущности") ;
- ВАЖНО!!!(идентификаторы извлеченных сущностей формируются по правилу ":label", где label - это словоформа, описывающая сущность (метка или имя сущности);
при этом, если словоформа длиннее, чем 40 символов, то она обрезается на 40-м символе; если словоформа состоит из нескольких слов, то она формируется как одно слово,
в котором вместо пробелов, точек подставляется нижнее подчеркивание (например ":оператор_отдела"));
- идентификатор_предиката берется из файла описывающего онтологию в соответствии с форматом ont_name:predicate_name;
- выведи ответ без каких-либо комментариев
ТЕКСТ:
```{chunk_content}```
ОНТОЛОГИЯ:
```{ontology}```
ПРИМЕР ОПИСАНИЯ извлеченной СУЩНОСТИ:
```
:подключение_к_Системе_МОНЕТА_РУ rdf:type comcore:Process ;
  rdfs:label "Подключение к Системе МОНЕТА.РУ"@ru .

:система_МОНЕТА_РУ rdf:type comcore:Resource ;
  rdfs:label "Система_МОНЕТА.РУ"@ru .

:клиент rdf:type comcore:Agent ;
  rdfs:label "Клиент" .
```
'''

  entities = llm_request(prompt_E, api_key)
  
  R_ontology = '''
@prefix comcore:  .
@prefix dc:  .
@prefix dcmtype:  .
@prefix dcterms:  .
@prefix owl:  .
@prefix rdf:  .
@prefix rdfs:  .
@prefix xsd:  .

comcore:isResultOf rdf:type owl:ObjectProperty;
  owl:inverseOf comcore:hasResult;
  rdf:type owl:IrreflexiveProperty;
  rdfs:label "результат"@ru;
  rdfs:label "result"@en;
  rdfs:domain comcore:Resource;
  rdfs:domain comcore:Event;
  rdfs:range comcore:Process;
  rdfs:comment """
Указывает что Событие или Ресурс (субъект) является результатом Процесса (объект).
Пример: Регистрация в форме может закончиться неуспешно если ИНН или Email уже зарегистрирован.
Субъекты: Успешная регистрация, ИНН уже зарегистрирован, Email уже зарегистрирован.""".

comcore:canBeResourceFor rdf:type owl:ObjectProperty;
  rdfs:label "может быть Ресурсом"@ru;
  rdfs:label "может использоваться как Ресурс в"@ru;
  rdfs:label "can be resource for"@en;
  rdfs:label "can be resource in"@en;
  rdfs:domain comcore:Resource;
  rdfs:range comcore:Process;
  rdfs:comment """Указывает что Ресурс (субъект) может использоваться в качестве входного Ресурса данного Процесса (объект). Например, ответ на секретный вопрос может использоваться при аутентификации""" .

comcore:isResourceOf rdf:type owl:ObjectProperty;
  owl:inverseOf comcore:hasResource;
  rdfs:label "является Ресурсом"@ru;
  rdfs:label "is Resource of"@en;
  rdfs:domain comcore:Resource;
  rdfs:range comcore:Process;
  rdfs:comment """Указывает что Ресурс (субъект) является необходимым входным Ресурсом данного Процесса (объект). Например, логин или пароль при аутентификации""" .

comcore:isActorOf rdf:type owl:ObjectProperty;
  owl:inverseOf comcore:hasActor;
  rdfs:label "является Актором"@ru;
  rdfs:label "is Actor of"@en;
  rdfs:comment "Указывает Агента (субъект), который в какой-то роли принимает участие в Процессе (объект)";
  rdfs:domain comcore:Agent;
  rdfs:range comcore:Process.

comcore:Initiates rdfs:subPropertyOf comcore:;
  rdfs:label "инициирует"@ru;
  rdfs:label "initiates"@en;
  rdfs:comment "Указывает что субъект инициирует данный Процесс.";
  rdfs:domain comcore:Event;
  rdfs:domain comcore:Agent;
  rdfs:range comcore:Process.

comcore:isPartOf rdfs:subPropertyOf dcterms:isPartOf;
  owl:inverseOf comcore:hasPart;
  rdf:type owl:IrreflexiveProperty;
  rdf:type owl:TransitiveProperty;
  rdfs:label "является частью"@ru;
  rdfs:label "is part of"@en;
  rdfs:domain comcore:Process;
  rdfs:domain comcore:Resource;
  rdfs:range comcore:Process;
  rdfs:range comcore:Resource;
  rdfs:comment "Указывает, что субъект является частью для данного объекта".

comcore:isResponsibleFor rdfs:subPropertyOf dcterms:contributor;
  rdfs:label "ответственный"@ru;
  rdfs:label "responsible"@en;
  rdfs:comment "Указывает, что Агент (субъект) отвечает за наличие/создание данного Ресурса (объект) или является ответственным за проведение Процесса";
  rdfs:domain comcore:Agent;
  rdfs:range comcore:Resource;
  rdfs:range comcore:Process.

comcore:relation rdfs:subPropertyOf dcterms:relation;
  rdf:type owl:IrreflexiveProperty;
  rdfs:label "связан с"@ru;
  rdfs:label "relation"@en;
  rdfs:comment "Указывает, что одна сущность (субъект) как-то связана с другой сущностью (объект). Отношение используется, если не подходят вышеперечисленные отношения".
'''

  prompt_R = f''' Извлеките из ТЕКСТА все отношения, описанными в ОНТОЛОГИИ, между ранее извлечепнными сущностями и сформируй описание триплетов с использованием RDF-модели в формате Turtle.
  ТРЕБОВАНИЯ:
  - сущность должна быть связана с другой сущностью отношением, описанным в ОНТОЛОГИИ:
  (идентификатор_сущности, идентификатор_предиката, идентификатор_сущности_2) ;
  - идентификатор_предиката берется из ОНТОЛОГИИ в соответствии с форматом ont_name:predicate_name;
  СУЩНОСТИ: {entities}
  ТЕКСТ: ```{chunk_content}```
  ОНТОЛОГИЯ отношений: ```{R_ontology}```
  ПРИМЕР ОПИСАНИЯ отношений:
  ```
  :подключение_к_Системе_МОНЕТА_РУ comcore:hasPart :проверка_готовности_функционала_сайта_ИМ ;
    comcore:hasPart :регистрация_в_Системе_МОНЕТА_РУ ;
    comcore:hasPart :заполнение_личного_кабинета .

  :система_МОНЕТА_РУ comcore:isResourceOf подключение_к_Системе_МОНЕТА_РУ .

  :клиент comcore:isResposibleFor :личный кабинет ;
    comcore:isActorOf :заполнение личного кабинета .
  ```
  '''

  return llm_request(prompt_R, api_key)

In [51]:
c = """
Подключение к Системе МОНЕТА.РУ - совокупность действий Получателя и/или Торговой площадки, в целях последующего осуществления Перевода с использованием Системы МОНЕТА.РУ:
 - регистрация Получателя в Системе МОНЕТА.РУ;
 - проверка готовности функционала сайта ИМ;
 - установление легитимности продаваемых сайтом товаров и услуг;
 - заполнение личного кабинета (ЛК) Клиента (предоставление в НКО данных о Получателе);
 - создание Бизнес - счета для Получателя в Системе МОНЕТА.РУ.
"""

API_KEY = "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"

In [ ]:
os.makedirs('./knowledges', exist_ok=True)

In [75]:
text = process_text_method_1(c, API_KEY)
print(text)
with open(f'./knowledges/chunk_test.ttl', 'w', encoding="windows-1251") as f:
    f.write(text)

Исходя из текста и онтологии, извлекаем следующие триплеты:

```turtle
# Отношения для главного процесса
:подключение_к_Системе_МОНЕТА_РУ comcore:hasPart :регистрация_Получателя_в_Системе_ ;
  comcore:hasPart :проверка_готовности_функционала_са ;
  comcore:hasPart :установление_легитимности_продава ;
  comcore:hasPart :заполнение_личного_кабинета_ЛК_Кл ;
  comcore:hasPart :создание_Бизнес_счета_для_Получат .

# Ресурсы для главного и дочерних процессов
:система_МОНЕТА_РУ comcore:isResourceOf :подключение_к_Системе_МОНЕТА_РУ ;
  comcore:isResourceOf :регистрация_Получателя_в_Системе_ ;
  comcore:isResourceOf :создание_Бизнес_счета_для_Получат .

# Агенты для главного процесса
:Получатель comcore:isActorOf :подключение_к_Системе_МОНЕТА_РУ .
:Торговая_площадка comcore:isActorOf :подключение_к_Системе_МОНЕТА_РУ .

# Ресурсы для дочерних процессов
:функционал_сайта_ИМ comcore:isResourceOf :проверка_готовности_функционала_са .
:товары_и_услуги comcore:isResourceOf :установление_легитимности_

In [67]:
for i in range(count_chunks):
    with open(f'./knowledges/chunk_{i:02d}.ttl', 'w', encoding="windows-1251") as f:
        f.write(text)